<a href="https://colab.research.google.com/github/juliayul1a/hmm/blob/main/Loading_Doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary:
We have been given a PDF with information about an insurance applicant.
We are tasked with leveraging OpenAI and Retrieval-Augmented Generation (RAG) to review the application and give an informed recommendation as to whether or not a company should provide insurance to this person.


# Assumptions:


*   The provided application is for life insurance.
*   While document interpretation should not be overfit to this particular example, it is likely that future applications will follow similar patterns.
*   This is whole life insurance and the **premiums do not change**. Premiums and other pricing considerations could be easily modified in future versions.
*   Approval should be granted when the company can be reasonably sure that they will not lose money on the policy.
*   The company will not lose money if the combination of applicant payments + modest interest returns on the cumulative payments is greater than the payout amount of the policy after n_years.
*   If accepted, the applicant will pay for the policy for the lesser of: their remaining lifetime or 100 years. Future versions apply additional predicted information leveraging attrition/churn modeling.
*   For the purposes of this exercise, the total amount paid by the applicant is equal to n_years * annual_premium, though future versions could be adapted to respond to different payment plans.


# Approach:


1.   Collect basic demographic information about the individual (age, sex)
2.   Parse the document for additional information about the individual's health and habits in order to determine an appropriate risk class
3.   Using this information and guesstimated weights, calculate the risk score and assign a risk class of premium, standard, or sub-standard to the individual
4.   With the individual's age, sex and risk class, leverage Pandas agents to lookup pre-loaded and processed data to answer the following:

*   **Question 1**: Is the applicant likely to pay an amount ≥ $250,000?

>*   **If no:** The company should reject the applicant.

>*   **If yes:** The company should find the greatest amount of insurance for which the applicant would be eligible.

*   **Question 2**:
For eligible applicants, what is the probability that they pay an amount ≥ each eligible tier of insurance?

# Outcome:
Provide the user with a summary of eligibility and final recommendations.



#Assets and Artifacts:

*   **[Billing codes](https://www.surgery.northwestern.edu/docs/cpt-codes.pdf):** Accessed via web 02.27.2024. Unmodified. Referenced via RAG.

*   **[Code Costs](https://www.cms.gov/research-statistics-data-and-systems/statistics-trends-and-reports/medicarefeeforsvcpartsab/downloads/level1charg13.pdf):** Accessed via web 02.27.2024. Slight modification and augmentation. Ultimately not leveraged in final solution.

*   **[Actuarial tables](https://www.ssa.gov/oact/STATS/table4c6.html):** Accessed via web 02.27.2024. Processing and augmentation applied. Referenced via Pandas agent.

*   **[Sample Policy Costs](https://www.policygenius.com/life-insurance/):** Accessed via web 02.27.2024. Heavily augmented via interpolation and extrapolation for use as sample data. Referenced via Pandas agent.

*   **[Sample Application]("fake-aps.pdf"):** Provided as part of challenge. Unmodified. Referenced via RAG.



# Loading Doc

In [ ]:
!pip install langchain openai weaviate-client PyMuPDF langchain_text_splitters --quiet

In [ ]:
from google.colab import userdata
ai_key = userdata.get('openAPIKey2')
import os
os.environ['OPENAI_API_KEY'] = ai_key

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

In [ ]:
loader = PyMuPDFLoader("sixfold_assets/fake-aps.pdf")

In [ ]:
documents = loader.load()

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [ ]:
markdown_document = str(documents)

headers_to_split_on = [
    ("Date: ", "Header 1"),
    ("Date: ", "Header 2"),
    ("Date: ", "Header 3"),
    ("Date: ", "Header 4"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
chunks = text_splitter.split_documents(documents)

In [ ]:
!pip install tiktoken --quiet

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

openai_api_key = OPENAI_API_KEY

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = md_header_splits,
    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    by_text = False
)

embedded weaviate is already listening on port 8079


In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
When answering, please provide only the answer and no additional text or information.
Here's an example: for the question, 'what is the applicant's name?' You should answer 'John Doe' instead of 'the applicant is named John Doe'
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nWhen answering, please provide only the answer and no additional text or information.\nHere's an example: for the question, 'what is the applicant's name?' You should answer 'John Doe' instead of 'the applicant is named John Doe'\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
openai_api_key = OPENAI_API_KEY
llm = ChatOpenAI(openai_api_key = openai_api_key, model_name="gpt-3.5-turbo-0125", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "What is the applicant name?"
rag_chain.invoke(query)

Embedded weaviate wasn't listening on ports http:8079 & grpc:50060, so starting embedded weaviate again
Started /root/.cache/weaviate-embedded: process ID 53873


'Sarah Miller'

In [ ]:
import datetime

In [ ]:
def binary_var(i):
  if i == 'no':
    return 0
  else:
    return 1

def date_var(i):
  date_str = i
  format_str = "%m/%d/%Y"
  i = datetime.datetime.strptime(date_str, format_str)
  return i

#QuantQs:
age_query = "What is the applicant's age?"
age = rag_chain.invoke(age_query)

#QualQs:
name_query = "what is the applicant's name?"
gender_query = "What is the applicant's gender? Please answer male or female."
smoker_query = "Is there any indication that the applicant is a smoker? Please answer yes or no."
driving_query = "Is there any indication that the applicant has a poor driving record or DUIs? Please answer yes or no."

#Qual - StringQs:
applicant_name = str(rag_chain.invoke(name_query)).lower()
gender = str(rag_chain.invoke(gender_query)).lower()

#Qual - BinaryQs:
smoker = binary_var(str(rag_chain.invoke(smoker_query)).lower())
driving = binary_var(str(rag_chain.invoke(driving_query)).lower())


# #DateQs:
visit_dates_query = f"looking through all of the provided documents, what are all of the dates that {applicant_name} saw a doctor?"
first_visit_query = f"looking through all of the provided documents, what was the first date that {applicant_name} saw a doctor?"
last_visit_query = f"looking through all of the provided documents, what was the last date that {applicant_name} saw a doctor?"
penultimate_visit_query = f"looking through all of the provided documents, what was the date immediately before {last_visit}?"

visit_dates = rag_chain.invoke(visit_dates_query).split(",")
first_visit = date_var(rag_chain.invoke(first_visit_query))
last_visit = date_var(rag_chain.invoke(last_visit_query))
penultimate_visit = date_var(rag_chain.invoke(penultimate_visit_query))

## Codes:
codes_by_date_query = f"what are the billing codes associated with each date in {visit_dates}? Please return the date and its corresponding code."
most_recent_code_query = f"what was the billing code for {applicant_name}'s last visit on {last_visit}?"

codes_by_date = rag_chain.invoke(codes_by_date_query)
most_recent_code = rag_chain.invoke(most_recent_code_query)

# Billing Code Review

In [ ]:
nu_loader = PyMuPDFLoader("https://www.surgery.northwestern.edu/docs/cpt-codes.pdf")

In [ ]:
nu_docs = nu_loader.load()


In [ ]:
from langchain.text_splitter import CharacterTextSplitter


text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(nu_docs)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

openai_api_key = OPENAI_API_KEY

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

nu_vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    by_text = False
)

nu_retriever = nu_vectorstore.as_retriever()

embedded weaviate is already listening on port 8079


In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """You are medical professional who is being asked to flag any high-risk codes found in a patient's files.
You will receive a list of CPT codes found in a patient's file.
To complete this task, you have been provided with a document which is a full catalog of CPT codes.
You should look up each code in the provided document.
If the code is not in the document, you can move on to the next code you received.
Based on the description you find in the document for the code and your medical expertise, please assign a rating on a scale of 0 through 5 with 0 being completely benign or positive (like an annual checkup) and 5 being imminent mortal danger.
In your answer, please provide each of the codes and their corresponding ratings in the format, code : rating
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are medical professional who is being asked to flag any high-risk codes found in a patient's files. \nYou will receive a list of CPT codes found in a patient's file. \nTo complete this task, you have been provided with a document which is a full catalog of CPT codes.\nYou should look up each code in the provided document.\nIf the code is not in the document, you can move on to the next code you received. \nBased on the description you find in the document for the code and your medical expertise, please assign a rating on a scale of 0 through 5 with 0 being completely benign or positive (like an annual checkup) and 5 being imminent mortal danger.\nIn your answer, please provide each of the codes and their corresponding ratings in the format, code : rating\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
openai_api_key = OPENAI_API_KEY
llm = ChatOpenAI(openai_api_key = openai_api_key, model_name="gpt-3.5-turbo-0125", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = f"{codes_by_date}"
code_answer = rag_chain.invoke(query)
print(answer)
print(type(answer))

99212 : 0
99395 : 0
99212 : 0
99396 : 0
<class 'str'>


In [ ]:
flagged_codes = int(rag_chain.invoke(f"what is the sum of the values assigned to each code in {code_answer}? Please provide only the number."))

In [ ]:
flagged_codes

0

In [ ]:
last_code = int(rag_chain.invoke(f"please reference {code_answer} and find the value after the colon for code {most_recent_code}. Please provide only the value and not the code in your answer."))

In [ ]:
last_code

0

# Weights & Class Assignments

In [ ]:
cur_year = datetime.datetime.today().year

### Assign Weights:

smoker_weight = -2
driving_weight = 0
if driving == 0:
  driving_weight = 0
else:
  driving_weight = -0.5
flagged_code_weight = -0.5
year_dif = cur_year-last_visit.year
# medical_recency_weight = year_dif
medical_recency_weight = 0
if year_dif == 0:
  medical_recency_weight += 1
else:
  medical_recency_weight += -0.5 * (year_dif - 1)
last_code_weight = 0
if last_code == 0:
  last_code += 2
  last_code_weight += 1
else:
  last_code_weight += -1

# Calculate Risk

risk_calc = (
    smoker * smoker_weight
    + driving * driving_weight
    + medical_recency_weight
    + last_code * last_code_weight
    + flagged_codes * flagged_code_weight
)

## Assign Risk Class
risk_class = ''
if risk_calc >= 1.5:
  risk_class += 'premium'
elif -1.5 <= risk_calc <= 1.4:
  risk_class += 'standard'
else:
  risk_class += 'sub-standard'

In [ ]:
risk_calc

0.5

# Actuary Data

**Primary tasks:**


*   Identify **n_years** of possible policy tenure
*   Iterate through range of **p_survival** for range(current_age - min_year) to return

**Key Inputs:**


*   **age**
*   **gender** (binary for the purposes of this exercise)

**Data & Other Assets:**


*   **[Actuarial tables](https://www.ssa.gov/oact/STATS/table4c6.html):** Accessed via web 02.27.2024. Processing and augmentation applied. Referenced via Pandas agent.
*   **[Tabula-Py](https://tabula-py.readthedocs.io/en/latest/)** Package for extracting tabular data from PDF
*   **[Pandas](https://pandas.pydata.org/docs/index.html)** For additional data processing

**Returns:**
DFs: [actuary_female, actuary_male]

**Ingest Actuarial Table Data**

In [ ]:
## Install & import functions for table ingestion and processing
!pip install -q tabula-py
import tabula
import pandas as pd

## Establish PDF Location
pdf_path = "sixfold_assets/actuarial_table.pdf"

## Process page 1 (slightly different processing steps than 2-3 & 4)
page1 = pd.DataFrame(tabula.read_pdf(pdf_path)[0])

## Define function for pages 2 & 3 which both require the same processing steps
def extract_page_data(pdf_path, page_number):
    df = pd.DataFrame(
        tabula.read_pdf(pdf_path, stream=True, pages=str(page_number), pandas_options={'header': None})[0]
    )
    df = df.iloc[4:]
    df.reset_index(drop=True, inplace=True)
    return df

## Apply function to process pages 2 & 3
page2 = extract_page_data(pdf_path, 2)
page3 = extract_page_data(pdf_path, 3)

##Define coordinates of table for page 4 (gathered using Mac Preview + Crop– see StackOF discussion here: http://tinyurl.com/mphkt9pb)
top = 13.77
left = 31.64
bottom = top + 524.28
right = left + 535.74

## Process page 4 (slightly different processing steps than 1 & 2-3 given need to define area)
page4 = pd.DataFrame(tabula.read_pdf(pdf_path, stream=True, pages ="4", area=[top, left, bottom, right], pandas_options={'header':None})[0])
page4 = page4.iloc[4:]
page4.reset_index(drop=True, inplace=True)


In [ ]:
##Clean up page 1 & assign headers
act_df = page1
actuarial = act_df.iloc[3:]
# actuarial.head()
actuarial.columns = ['age', 'male_p_death', 'male_num_lives', 'male_life_expt',
              'female_p_death', 'female_num_lives', 'female_life_expt']
actuarial.reset_index(drop=True, inplace=True)

# Display the modified dataframe
actuarial.head(110)

,age,male_p_death,male_num_lives,male_life_expt,female_p_death,female_num_lives,female_life_expt
0,0,0.005837,"100,000",74.12,0.004907,"100,000",79.78
1,1,0.000410,"99,416",73.55,0.000316,"99,509",79.17
2,2,0.000254,"99,376",72.58,0.000196,"99,478",78.19
3,3,0.000207,"99,350",71.60,0.000160,"99,458",77.21
4,4,0.000167,"99,330",70.62,0.000129,"99,442",76.22
5,5,0.000141,"99,313",69.63,0.000109,"99,430",75.23
6,6,0.000123,"99,299",68.64,0.000100,"99,419",74.24
7,7,0.000113,"99,287",67.65,0.000096,"99,409",73.25
8,8,0.000108,"99,276",66.65,0.000092,"99,399",72.25
9,9,0.000114,"99,265",65.66,0.000089,"99,390",71.26


In [ ]:
## Shorten names for easier concatenation readability
p1 = actuarial
p2 = page2
p3 = page3
p4 = page4

## Assign p1 columns to each additional page

dataframes = [p2, p3, p4]

for df in dataframes:
    df.columns = p1.columns


## Concatenate pages into one table (120 x 7)

actuary = pd.concat([p1, p2, p3, p4], axis=0,ignore_index=True)
actuary.apply(pd.to_numeric, errors='coerce')
actuary.head()
# cols = ['col1', 'col2', 'col3']
# data[cols] = data[cols].apply(pd.to_numeric, errors='coerce', axis=1)

,age,male_p_death,male_num_lives,male_life_expt,female_p_death,female_num_lives,female_life_expt
0,0,0.005837,"100,000",74.12,0.004907,"100,000",79.78
1,1,0.000410,"99,416",73.55,0.000316,"99,509",79.17
2,2,0.000254,"99,376",72.58,0.000196,"99,478",78.19
3,3,0.000207,"99,350",71.60,0.000160,"99,458",77.21
4,4,0.000167,"99,330",70.62,0.000129,"99,442",76.22


In [ ]:
len(actuary)

120

**Give additional calculations and clarity to Actuary Table**

In [ ]:
# Initialize p_survive columns for male and female
actuary.insert(loc= 2, column='male_p_survive', value=[0 for i in range(actuary.shape[0])])
actuary.insert(loc= 6, column='female_p_survive', value=[0 for i in range(actuary.shape[0])])

# Set both p_death columns to numeric for calculations
actuary['male_p_death'] = pd.to_numeric(actuary.male_p_death)
actuary['female_p_death'] = pd.to_numeric(actuary.female_p_death)


# Apply calc 1-p_death to each row in actuary for male and female to calculate p_survive
for i in range(len(actuary)):
    p_death = actuary.loc[i, 'male_p_death']
    actuary.loc[i, 'male_p_survive'] = 1 - p_death

for i in range(len(actuary)):
    p_death = actuary.loc[i, 'female_p_death']
    actuary.loc[i, 'female_p_survive'] = 1 - p_death

actuary.head()


,age,male_p_death,male_p_survive,male_num_lives,male_life_expt,female_p_death,female_p_survive,female_num_lives,female_life_expt
0,0,0.005837,0.994163,"100,000",74.12,0.004907,0.995093,"100,000",79.78
1,1,0.000410,0.999590,"99,416",73.55,0.000316,0.999684,"99,509",79.17
2,2,0.000254,0.999746,"99,376",72.58,0.000196,0.999804,"99,478",78.19
3,3,0.000207,0.999793,"99,350",71.60,0.000160,0.999840,"99,458",77.21
4,4,0.000167,0.999833,"99,330",70.62,0.000129,0.999871,"99,442",76.22


In [ ]:
actuary_male = actuary[['age', 'male_p_death', 'male_p_survive', 'male_num_lives', 'male_life_expt']].copy()
actuary_female = actuary[['age', 'female_p_death', 'female_p_survive', 'female_num_lives', 'female_life_expt']].copy()


In [ ]:
## Drop gender flags from column names for both dfs & write out words for clarity and readability

actuary_male.columns = ['age', 'probability_death', 'probability_survive', 'num_lives', 'life_expectancy']
actuary_female.columns = actuary_male.columns

#Confirm change with 2nd table:
actuary_female.head()

,age,probability_death,probability_survive,num_lives,life_expectancy
0,0,0.004907,0.995093,"100,000",79.78
1,1,0.000316,0.999684,"99,509",79.17
2,2,0.000196,0.999804,"99,478",78.19
3,3,0.000160,0.999840,"99,458",77.21
4,4,0.000129,0.999871,"99,442",76.22


In [ ]:
##Quick sanity check
print(len(actuary_female), len(actuary_male))

120 120


**END ACTUARY BLOCK**

DFs = [actuary_female, actuary_male]

# Cost Data by Gender and Age


**Primary tasks:**


*   Serve as reference for annual premium costs based on the applicant's age, sex and risk class.

**Key Inputs:**


*   **age**
*   **gender** (Binary for the purposes of this exercise)
*   **risk_class**

**Data & Other Assets:**


*   Two instances of sample cost data for whole life insurance policies

**[Sample Policy Costs](https://www.policygenius.com/life-insurance/)**: Accessed via web 02.27.2024. Heavily augmented via interpolation and extrapolation for use as sample data. Referenced via Pandas agent.

**Returns:**

DFs: [female_costs, male_costs]

In [ ]:
male_costs = pd.read_csv('sixfold_assets/male_costs.csv')
female_costs = pd.read_csv('sixfold_assets/female_costs.csv')
female_costs.head()


,Age,Gender,250_monthly_cost,250_annual_cost,500_monthly_cost,500_annual_cost,1m_monthly_cost,1m_annual_cost
0,20,Female,$149.00,"$1,788.00",$293.00,"$3,516.00",$572.00,"$6,864.00"
1,21,Female,$155.40,"$1,864.80",$305.80,"$3,669.60",$597.50,"$7,170.00"
2,22,Female,$161.80,"$1,941.60",$318.60,"$3,823.20",$623.00,"$7,476.00"
3,23,Female,$168.20,"$2,018.40",$331.40,"$3,976.80",$648.50,"$7,782.00"
4,24,Female,$174.60,"$2,095.20",$344.20,"$4,130.40",$674.00,"$8,088.00"


# Pandas Agents

In [ ]:
!pip install langchain_experimental --quiet
from langchain.llms import OpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent

**Actuary Agent**

For simplicty, identify the gender of the applicant and assign a table accordingly.

Pass that table to the agent.

Ask the agent to query the table for the desired information.

**Returns:** num_years (life_expectancy given age)

In [ ]:
# Assign either male or female data to actuary_table based on applicant context
actuary_table = []
if gender == 'female':
  actuary_table = actuary_female
else:
  actuary_table = actuary_male

In [ ]:
# Create actuary_agent
actuary_agent = create_pandas_dataframe_agent(OpenAI(temperature=0), actuary_table, verbose=False)

In [ ]:
# Identify life_expectancy given age and set to int
life_expectancy = int(float(actuary_agent.run(f"What is the life expectancy for someone who is {age} years old? Please reply with a number.")))
num_years = life_expectancy

**Cost Agent**

For simplicty, identify the gender of the applicant and assign a table accordingly. This will prevent us from surpassing our token window.

Pass that table to the agent.

Ask the agent to query the table for the desired information.

**Returns:** annual_cost for a 250k, 500k and 1M policy

In [ ]:
# Assign either male or female data to cost_table based on applicant context

cost_table = []
if gender == 'female':
  cost_table = female_costs
else:
  cost_table = male_costs

In [ ]:
cost_table.head(5)

,Age,Gender,250_monthly_cost,250_annual_cost,500_monthly_cost,500_annual_cost,1m_monthly_cost,1m_annual_cost
0,20,Female,$149.00,"$1,788.00",$293.00,"$3,516.00",$572.00,"$6,864.00"
1,21,Female,$155.40,"$1,864.80",$305.80,"$3,669.60",$597.50,"$7,170.00"
2,22,Female,$161.80,"$1,941.60",$318.60,"$3,823.20",$623.00,"$7,476.00"
3,23,Female,$168.20,"$2,018.40",$331.40,"$3,976.80",$648.50,"$7,782.00"
4,24,Female,$174.60,"$2,095.20",$344.20,"$4,130.40",$674.00,"$8,088.00"


In [ ]:
# age = int(age)
# # Create cost_agent
# cost_agent = create_pandas_dataframe_agent(OpenAI(temperature=0), cost_table, verbose=False)

In [ ]:
# # Given age, identify the annual premium for $250, $500 and $1M policies
# annual_premium_250 = actuary_agent.run(f"what is 250_annual_cost for someone who is {age} years old?")
# annual_premium_250

'Agent stopped due to iteration limit or time limit.'

In [ ]:
## Cost agent was unceremoniously fired.

In [ ]:
# Filter cost_table by age
cost_filtered = cost_table[cost_table['Age'] == age]
import re

# Assigning to unique variables
annual_premium_250 = re.sub(r'[^\d.]','', cost_filtered["250_annual_cost"].values[0])
annual_premium_500 = re.sub(r'[^\d.]','', cost_filtered["500_annual_cost"].values[0])
annual_premium_1M = re.sub(r'[^\d.]','', cost_filtered["1m_annual_cost"].values[0])




In [ ]:
def risk_adjustment(i):
  if risk_class == 'premium':
    i = i * .95
  elif risk_class == 'sub-standard':
    i = i * 1.05
  else:
    i = i
  return i

In [ ]:
annual_premium_250 = risk_adjustment(annual_premium_250)
annual_premium_500 = risk_adjustment(annual_premium_500)
annual_premium_1M = risk_adjustment(annual_premium_1M)

# Key Info

In [ ]:
key_info = (age, gender, risk_class, num_years)

In [ ]:
key_info

('39', 'female', 'standard')

In [ ]:
policy_amounts = [250000, 500000, 1000000]


list

# Should Approve?

In [ ]:
num_years = int(num_years)
annual_premium = float(annual_premium_250)
annual_premium = int(annual_premium)
lifetime_premium = num_years * annual_premium
print(lifetime_premium)
### Remember to cap out years at 100

153132


In [ ]:
##Assume modest interest earned on policy premiums
def calculate_interest(initial_investment, years, average_apy):

  # Convert APY to decimal format
  apy_decimal = average_apy / 100

  # Calculate the total amount invested
  total_invested = initial_investment * (years + 1)

  # Calculate the total interest earned
  total_interest = total_invested * apy_decimal * years

  return total_interest

# Example usage
initial_investment = annual_premium
years = num_years
average_apy = 2.1

total_interest = calculate_interest(initial_investment, years, average_apy)

print(f"Total interest earned: ${total_interest:.2f}")


Total interest earned: $138278.20


In [ ]:
total_dollars = lifetime_premium + total_interest

In [ ]:
lowest_threshold = policy_amounts[0]
lowest_threshold = int(lowest_threshold)

In [ ]:
should_insure = 0
if lowest_threshold > total_dollars:
  should_insure += 0
else:
  should_insure += 1

In [ ]:
should_insure

1

In [ ]:
decision = ''
if should_insure == 1:
  decision = "should be insured"
else:
  decision = "should not be approved for insurance at this time."

# If approved, at what amount?

In [ ]:
## Find the amounts for 250, 500 and 1M:


##250
annual_premium_250 = float(annual_premium_250)
annual_premium_250 = int(annual_premium_250)
lifetime_premium_250 = num_years * annual_premium_250

initial_investment = annual_premium_250
years = num_years
average_apy = 2.1

total_interest_250 = calculate_interest(initial_investment, years, average_apy)

total_dollars_250 = total_interest_250 + lifetime_premium_250

##500
annual_premium_500 = float(annual_premium_500)
annual_premium_500 = int(annual_premium_500)
lifetime_premium_500 = num_years * annual_premium_500

initial_investment = annual_premium_500
years = num_years
average_apy = 2.1

total_interest_500 = calculate_interest(initial_investment, years, average_apy)

total_dollars_500 = total_interest_500 + lifetime_premium_500

##1M
annual_premium_1M = float(annual_premium_1M)
annual_premium_1M = int(annual_premium_1M)
lifetime_premium_1M = num_years * annual_premium_1M

initial_investment = annual_premium_1M
years = num_years
average_apy = 2.1

total_interest_1M = calculate_interest(initial_investment, years, average_apy)

total_dollars_1M = total_interest_1M + lifetime_premium_1M

In [ ]:
approved_amount = 0
##Check 1M First
if total_dollars_1M >= policy_amounts[2]:
  approved_amount += policy_amounts[2]
  annual_premium = annual_premium_1M
  lifetime_premium = lifetime_premium_1M
  total_interest = total_interest_1M
  total_dollars = total_dollars_1M
##Check 500k Second
elif total_dollars_500 >= policy_amounts[1]:
  approved_amount += policy_amounts[1]
  annual_premium = annual_premium_500
  lifetime_premium = lifetime_premium_500
  total_interest = total_interest_500
  total_dollars = total_dollars_500
else:
  approved_amount += policy_amounts[0]
  annual_premium = annual_premium_250
  lifetime_premium = lifetime_premium_250
  total_interest = total_interest_250
  total_dollars = total_dollars_250


total_return = approved_amount - total_dollars

In [ ]:
approved_amount

1000000

# Conclusion

In [ ]:
if should_insure == 1:
  print(f"{patient_name}, a {age} year old {gender}, {decision} up to an amount of ${approved_amount} at {risk_class} rates.\n The annual premium for this policy is expected to be around ${annual_premium} for a term of approximately {num_years} years.\n If properly invested, the premium for this policy is projected to return ${total_interest} in interest during the term period based on a rate of {average_apy}% APY.")
else:
  print(f"{patient_name}, a {age} year old {gender}, {decision}")

Sarah Miller, a 39 year old female, should be insured up to an amount of $1000000 at standard rates.
 The annual premium for this policy is expected to be around $14265 for a term of approximately 42 years.
 If properly invested, the premium for this policy is projected to return $541014.39 in interest during the term period based on a rate of 2.1% APY.
